In [254]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [255]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [256]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [257]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
digitalalliance_video
/02_regular_reports


In [258]:
data_link = download_response.content

In [259]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [260]:
sheet_names.sheet_names

['Sheet1']

In [261]:
report_type = 'test'

In [316]:
cols_range = 'B:K' # задаем диапазон полей, которые нам нужны


df = pd.read_excel(BytesIO(data_link), sheet_name='Sheet1', usecols=cols_range)
start_index = list(df[df['Unnamed: 1'].str.lower()=='название рм'].index)[0] # забираем индекс строки, где содержаться заголовки
df.columns = df.iloc[start_index].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
df = df.rename(columns={'название рм': 'product', 'дата': 'date', 'показы': 'impressions', 'клики': 'clicks',
                       'досмотры рекламы до 25%': '25', 'досмотры рекламы до 50%': '50', 
                       'досмотры рекламы до 75%': '75', 'досмотры рекламы до 100%': '100', 'уники': 'reach'})
df = df[['product', 'date', 'impressions', 'clicks',  '25', '50', '75', '100', 'reach']]

df = df.fillna(0)
end_index = list(df[df['date']==0].index)[1]
df = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны


df['source'] = 'digitalalliance' #добавляем название источника
df['date'] = pd.to_datetime(df['date']).dt.date  # приводим в формат даты
df['product'] = df['product'].str.lower().str.strip()
df['report_type'] = report_type
df['format_type'] = 'video' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)

In [309]:
end_index

65

In [317]:
df

1,product,date,impressions,clicks,25,50,75,100,reach,source,report_type,format_type
2,adv_15_sec_olv_web_edit.mp4,2024-08-26,10034,32,9835,9711,9645,9599,7227,digitalalliance,test,video
3,adv_15_sec_olv_web_edit.mp4,2024-08-27,22046,36,21671,21480,21367,21267,14596,digitalalliance,test,video
4,adv_15_sec_olv_web_edit.mp4,2024-08-28,17797,41,17477,17331,17224,17158,11724,digitalalliance,test,video
5,adv_15_sec_olv_web_edit.mp4,2024-08-29,13474,22,13209,13096,13029,12967,9101,digitalalliance,test,video
6,adv_15_sec_olv_web_edit.mp4,2024-08-30,11283,14,11075,11003,10957,10911,7875,digitalalliance,test,video
7,adv_15_sec_olv_web_edit.mp4,2024-08-31,2866,4,2815,2799,2793,2786,1925,digitalalliance,test,video
8,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-26,10192,10,10040,10040,10004,9945,7275,digitalalliance,test,video
9,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-27,21891,14,21650,21650,21572,21460,14599,digitalalliance,test,video
10,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-28,17606,19,17398,17398,17339,17276,11606,digitalalliance,test,video
11,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-29,13652,11,13464,13464,13399,13340,9169,digitalalliance,test,video


In [ ]:
# создаем список с названиями полей
cols_name = ['product', 'date', 'impressions', 'clicks',  '25', '50', '75', '100', 'reach']

C:\Users\o.bogomolov\AppData\Local\Temp\3\ipykernel_18716\2900375399.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[0][0]


nan

In [290]:
df

1,product,date,impressions,clicks,25,50,75,100,reach
2,adv_15_sec_olv_web_edit.mp4,2024-08-26 00:00:00,10034,32,9835,9711,9645,9599,7227
3,adv_15_sec_olv_web_edit.mp4,2024-08-27 00:00:00,22046,36,21671,21480,21367,21267,14596
4,adv_15_sec_olv_web_edit.mp4,2024-08-28 00:00:00,17797,41,17477,17331,17224,17158,11724
5,adv_15_sec_olv_web_edit.mp4,2024-08-29 00:00:00,13474,22,13209,13096,13029,12967,9101
6,adv_15_sec_olv_web_edit.mp4,2024-08-30 00:00:00,11283,14,11075,11003,10957,10911,7875
7,adv_15_sec_olv_web_edit.mp4,2024-08-31 00:00:00,2866,4,2815,2799,2793,2786,1925
8,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-26 00:00:00,10192,10,10040,10040,10004,9945,7275
9,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-27 00:00:00,21891,14,21650,21650,21572,21460,14599
10,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-28 00:00:00,17606,19,17398,17398,17339,17276,11606
11,adv_6_sec_v1_edit_olv_web_edit.mp4,2024-08-29 00:00:00,13652,11,13464,13464,13399,13340,9169


In [245]:
df = df[['product', 'date', 'impressions', 'clicks', '25', '50', '75', '100', 'reach']]
df = df.fillna(0)

end_index = list(df[df['date']==0].index)[0]
df = df.iloc[:end_index] # оставляем строки с данными, которые нам нужны

df['source'] = 'digitalalliance' #добавляем название источника
df['date'] = pd.to_datetime(df['date']).dt.date  # приводим в формат даты
df['product'] = df['product'].str.lower()
df['report_type'] = report_type
df['format_type'] = 'video' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)

In [ ]:
# источник digitalalliance
# тип отчета Видео реклама
# Функция для обработки статистики по дням для обычных размещений 
# Статистика в разбивке по дням находится на листе Sheet1
# Название креатива находится в поле В (Название РМ)
# Охват находится в поле К (Уники)

def get_digitalalliance_video_report(data_link, report_type):
    cols_range = 'B:K' # задаем диапазон полей, которые нам нужны

    # создаем список с названиями полей
    cols_name = ['product', 'date', 'impressions', 'clicks', 'ctr', '25', '50', '75', '100', 'reach']
    df = pd.read_excel(BytesIO(data_link), sheet_name='Sheet1', skiprows=2, usecols=cols_range, names=cols_name)
    df = df[['product', 'date', 'impressions', 'clicks', '25', '50', '75', '100', 'reach']]
    df = df.fillna(0)
    
    end_index = list(df[df['date']==0].index)[0]
    df = df.iloc[:end_index] # оставляем строки с данными, которые нам нужны
    
    df['source'] = 'digitalalliance' #добавляем название источника
    df['date'] = pd.to_datetime(df['date']).dt.date  # приводим в формат даты
    df['product'] = df['product'].str.lower()
    df['report_type'] = report_type
    df['format_type'] = 'video' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)

    return df